In [1]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

import torch 
from feature_processor import FeatureProcessor

from candidate_generator import CandidateGenerator
from validate_model import validate_model_with_features
from ncf import NCF
from cold_start import ColdStartRecommender

np.random.seed(123)


In [2]:
ratings = pd.read_csv("../two_towers/data/ml-1m/ratings.dat", sep="::", header=None)
ratings.columns = ["user_id", "movie_id", "rating", "timestamp"]

movies = pd.read_csv("../two_towers/data/ml-1m/movies.dat", sep="::", header=None)
movies.columns = ["movie_id", "title", "genres"]

users = pd.read_csv("../two_towers/data/ml-1m/users.dat", sep="::", header=None)
users.columns = ["user_id", "gender", "age", "occupation", "zip_code"]


/tmp/ipykernel_97100/496125939.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings = pd.read_csv("../two_towers/data/ml-1m/ratings.dat", sep="::", header=None)
/tmp/ipykernel_97100/496125939.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies = pd.read_csv("../two_towers/data/ml-1m/movies.dat", sep="::", header=None)
/tmp/ipykernel_97100/496125939.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


In [3]:
rand_userIds = np.random.choice(ratings['user_id'].unique(),
                               size=int(len(ratings['user_id'].unique())*0.1),
                               replace=False)
ratings = ratings.loc[ratings['user_id'].isin(rand_userIds)]

print(ratings.head())

     user_id  movie_id  rating  timestamp
799       10      2622       5  978228212
800       10       648       4  978224925
801       10      2628       3  978228408
802       10      3358       5  978226378
803       10      3359       3  978227125


In [4]:
# Initialize feature processor and prepare features
print("Preparing user and movie features...")

# Initialize feature processor
feature_processor = FeatureProcessor()

# Prepare user features
user_features_df = feature_processor.prepare_user_features(users)

# Prepare movie features  
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
movie_embeddings = feature_processor.prepare_movie_features(movies, device=device)

print(f"\nFeature preparation complete!")
print(f"User feature dimension: {feature_processor.user_feature_dim}")
print(f"Movie feature dimension: {feature_processor.movie_feature_dim}")


Preparing user and movie features...
Preparing user features...
User features shape: (6040, 30)
User feature columns: ['user_id', 'gender', 'age_1', 'age_18', 'age_25', 'age_35', 'age_45', 'age_50', 'age_56', 'occ_0', 'occ_1', 'occ_2', 'occ_3', 'occ_4', 'occ_5', 'occ_6', 'occ_7', 'occ_8', 'occ_9', 'occ_10', 'occ_11', 'occ_12', 'occ_13', 'occ_14', 'occ_15', 'occ_16', 'occ_17', 'occ_18', 'occ_19', 'occ_20']
User feature dtypes:
user_id    float64
gender     float64
age_1      float64
age_18     float64
age_25     float64
age_35     float64
age_45     float64
age_50     float64
age_56     float64
occ_0      float64
occ_1      float64
occ_2      float64
occ_3      float64
occ_4      float64
occ_5      float64
occ_6      float64
occ_7      float64
occ_8      float64
occ_9      float64
occ_10     float64
occ_11     float64
occ_12     float64
occ_13     float64
occ_14     float64
occ_15     float64
occ_16     float64
occ_17     float64
occ_18     float64
occ_19     float64
occ_20     float64


/home/kevin/miniconda3/envs/transformers/lib/python3.13/site-packages/torch/cuda/__init__.py:174: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


Encoding movie titles...
Encoding movie genres...
Title embeddings shape: torch.Size([3883, 384])
Genre embeddings shape: torch.Size([3883, 384])
Combined movie embeddings shape: torch.Size([3883, 768])
Normalizing movie embeddings...
Original embeddings - mean: 0.0010, std: 0.0510
Normalized embeddings - mean: 0.0007, std: 0.0361
Movie feature dimension: 768

Feature preparation complete!
User feature dimension: 29
Movie feature dimension: 768


In [5]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Index: 97208 entries, 799 to 998118
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   user_id    97208 non-null  int64
 1   movie_id   97208 non-null  int64
 2   rating     97208 non-null  int64
 3   timestamp  97208 non-null  int64
dtypes: int64(4)
memory usage: 3.7 MB


In [6]:
ratings['rank_latest'] = ratings.groupby(['user_id'])['timestamp'] \
                                .rank(method = 'first',ascending=False)

train_ratings = ratings[ratings['rank_latest'].isin([1,2])]
validation_ratings = ratings[ratings['rank_latest'] == 1]
test_ratings = ratings[ratings['rank_latest'] == 2]

# drop columns that we no Longer need 
train_ratings = train_ratings[['user_id', 'movie_id', 'rating']]
test_ratings = test_ratings[['user_id','movie_id','rating']]

In [7]:
num_users = ratings['user_id'].max()+1
num_items = ratings['movie_id'].max()+1

all_movieIds = ratings['movie_id'].unique()

In [8]:


candidate_gen = CandidateGenerator(ratings, movies, all_movieIds)

def generate_candidates(user_id, method="hybrid", num_candidates=100):
    """Backward compatible interface"""
    return candidate_gen.generate_candidates(user_id, method, num_candidates)

In [9]:
# Precompute validation candidates for faster validation
def precompute_validation_candidates(validation_ratings, candidate_method="hybrid", num_candidates=100):
    """
    Precompute candidates for all validation users to speed up validation
    
    Returns:
        dict: {user_id: [candidate_items]}
    """
    validation_users = validation_ratings['user_id'].unique()
    precomputed_candidates = {}
    
    print(f"Precomputing candidates for {len(validation_users)} validation users...")
    
    for user_id in tqdm(validation_users, desc="Precomputing candidates"):
        candidates = generate_candidates(user_id, method=candidate_method, num_candidates=num_candidates)
        precomputed_candidates[user_id] = candidates
    
    print(f"Precomputed candidates for {len(precomputed_candidates)} users")
    return precomputed_candidates

# Precompute candidates before training
validation_candidates = precompute_validation_candidates(validation_ratings, candidate_method="hybrid", num_candidates=100)

Precomputing candidates for 604 validation users...


Precomputing candidates:   0%|          | 0/604 [00:00<?, ?it/s]

Precomputed candidates for 604 users


In [10]:
sampling_strategies = [
    ("unique_per_user", "hybrid"),
]

results = {}

for sampling_strategy, neg_method in sampling_strategies:
    print(f"\n{'='*70}")
    print(f"Training with {sampling_strategy.upper()} sampling + {neg_method.upper()} negatives")
    print(f"{'='*70}")
    
    # Create optimized model with CORRECT feature dimensions
    model = NCF(
        user_feature_dim=feature_processor.user_feature_dim,
        movie_feature_dim=feature_processor.movie_feature_dim,
        ratings=train_ratings, 
        feature_processor=feature_processor,
        candidate_generator=candidate_gen,
        negative_method=neg_method, 
        sampling_strategy=sampling_strategy
    )

    # Set up training parameters
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters())
    dataloader = model.get_dataloader(batch_size=512, num_workers=4, num_negatives=4)

    # Training loop
    num_epochs = 30
    
    epoch_results = []
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        total_loss = 0
        num_batches = 0
        
        for batch in tqdm(dataloader, desc=f"Training Epoch {epoch+1}/{num_epochs}"):
            # Move batch to device
            user_input, item_input, labels = [x.to(device) for x in batch]
            batch_device = (user_input, item_input, labels)
            
            # Zero gradients
            optimizer.zero_grad()
            
            # Forward pass and compute loss
            loss = model.compute_loss(batch_device)
            
            # Backward pass
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            num_batches += 1
        
        avg_loss = total_loss / num_batches
        
        # Validation phase
        model.eval()
        with torch.no_grad():
            hit_ratio, mrr, mean_rank = validate_model_with_features(
                model, validation_ratings, validation_candidates, device, 
                total_users_to_test=20, k=10
            )
        
        epoch_results.append({
            'epoch': epoch + 1,
            'train_loss': avg_loss,
            'hit_ratio': hit_ratio,
            'mean_rank': mean_rank
        })
        
        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"  Training Loss: {avg_loss:.4f}")
        print(f"  Hit Ratio @ 10: {hit_ratio:.3f}")
        print(f"  Mean Rank: {mean_rank:.1f}")
        print("-" * 50)

    # Store results
    strategy_name = f"{sampling_strategy}_{neg_method}"
    results[strategy_name] = epoch_results
    
    print(f"Training with {strategy_name} completed!")
    print(f"Final Hit Ratio @ 10: {hit_ratio:.3f}")
    print(f"Final Mean Rank: {mean_rank:.1f}")

# Summary of results
print(f"\n{'='*70}")
print("FINAL RESULTS SUMMARY")
print(f"{'='*70}")

for strategy_name, epoch_results in results.items():
    final_result = epoch_results[-1]
    print(f"{strategy_name:30} | Hit Ratio: {final_result['hit_ratio']:.3f} | Mean Rank: {final_result['mean_rank']:.1f}")

print("\nAll experiments completed!")



Training with UNIQUE_PER_USER sampling + HYBRID negatives
Generating training dataset with hybrid negative sampling
Sampling strategy: unique_per_user


Processing users:   0%|          | 0/604 [00:00<?, ?it/s]

Processing users: 100%|██████████| 604/604 [00:02<00:00, 244.42it/s]


Generated 6040 samples (1208 positive, 4832 negative)
Negative-to-positive ratio: 4.00


Training Epoch 1/30:   0%|          | 0/12 [00:00<?, ?it/s]

...

Training Epoch 30/30:   0%|          | 0/12 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

User 5608, Movie 3160: Score=0.827118456363678, Rank=15/101
User 3912, Movie 318: Score=0.05971883237361908, Rank=56/101
User 1878, Movie 1920: Score=0.8863867521286011, Rank=6/101


100%|██████████| 20/20 [00:00<00:00, 40.57it/s]


Validation Summary:
Total test cases: 20
Skipped cases: 0
Valid cases processed: 20
Hit Ratio @ 10: 0.400
Mean Rank: 17.3
MRR: 0.113
Epoch 30/30
  Training Loss: 0.2203
  Hit Ratio @ 10: 0.400
  Mean Rank: 17.3
--------------------------------------------------
Training with unique_per_user_hybrid completed!
Final Hit Ratio @ 10: 0.400
Final Mean Rank: 17.3

FINAL RESULTS SUMMARY
unique_per_user_hybrid         | Hit Ratio: 0.400 | Mean Rank: 17.3

All experiments completed!


In [11]:
print("="*80)
print("MODEL SAVE AND LOAD DEMONSTRATION")
print("="*80)

# Save the trained model
model_save_path = "models/ncf_trained_model.pth"
print(f"\n1. SAVING MODEL")
print("-" * 50)

# Create models directory if it doesn't exist
import os
os.makedirs("models", exist_ok=True)

# Save the trained model
model.save_weights(model_save_path)

MODEL SAVE AND LOAD DEMONSTRATION

1. SAVING MODEL
--------------------------------------------------
Model weights saved to models/ncf_trained_model.pth


In [12]:
print("Testing model performance on test set...")

def precompute_test_candidates(test_ratings, candidate_method="hybrid", num_candidates=100):
    test_users = test_ratings['user_id'].unique()
    precomputed_candidates = {}
    
    print(f"Precomputing candidates for {len(test_users)} test users...")
    
    for user_id in tqdm(test_users, desc="Precomputing test candidates"):
        candidates = generate_candidates(user_id, method=candidate_method, num_candidates=num_candidates)
        precomputed_candidates[user_id] = candidates
    
    return precomputed_candidates

test_candidates = precompute_test_candidates(test_ratings, candidate_method="hybrid", num_candidates=100)

model.eval()
with torch.no_grad():
    print(f"Evaluating on test set...")
    test_hit_ratio, test_mrr, test_mean_rank = validate_model_with_features(
        model, test_ratings, test_candidates, device, 
        total_users_to_test=50, k=10
    )

print(f"TEST SET RESULTS:")
print(f"Hit Ratio @ 10: {test_hit_ratio:.3f}")
print(f"Mean Rank: {test_mean_rank:.1f}")
print(f"MRR: {test_mrr:.3f}")


Testing model performance on test set...
Precomputing candidates for 604 test users...


Precomputing test candidates:   0%|          | 0/604 [00:00<?, ?it/s]

Evaluating on test set...


 10%|█         | 5/50 [00:00<00:01, 42.05it/s]

User 2898, Movie 2502: Score=0.9365142583847046, Rank=3/101
User 4689, Movie 3793: Score=0.8687427639961243, Rank=5/101
User 3138, Movie 1580: Score=0.14046785235404968, Rank=34/101


100%|██████████| 50/50 [00:01<00:00, 42.05it/s]


Validation Summary:
Total test cases: 50
Skipped cases: 0
Valid cases processed: 50
Hit Ratio @ 10: 0.540
Mean Rank: 16.0
MRR: 0.190
TEST SET RESULTS:
Hit Ratio @ 10: 0.540
Mean Rank: 16.0
MRR: 0.190


In [13]:
# Demonstrate loading complete model using class method
print(f"\n4. LOADING COMPLETE MODEL (CLASS METHOD)")
print("-" * 50)

loaded_model = NCF.load_model(
    filepath=model_save_path,
    ratings=train_ratings,
    feature_processor=feature_processor,
    candidate_generator=candidate_gen
)



4. LOADING COMPLETE MODEL (CLASS METHOD)
--------------------------------------------------
Complete model loaded successfully from models/ncf_trained_model.pth


In [14]:

# Initialize the cold start recommender with the trained model
cold_start_recommender = ColdStartRecommender(
    trained_model=loaded_model,
    feature_processor=feature_processor,
    candidate_generator=candidate_gen,
    movies_df=movies
)

print("Cold Start Recommender initialized successfully!")


Cold Start Recommender initialized successfully!


In [16]:
# DEMONSTRATION: Cold Start Recommendation Examples

print("="*80)
print("COLD START RECOMMENDATION SYSTEM DEMONSTRATION")
print("="*80)

# Example 1: Pure Cold Start - New user with only demographics
print("\n1. PURE COLD START SCENARIO")
print("-" * 50)

new_user_demographics = {
    'gender': 'M',     # Male
    'age': 25,         # 25 years old  
    'occupation': 4    # College/grad student (based on MovieLens occupation codes)
}

print(f"New User Demographics: {new_user_demographics}")

# Get recommendations without any ratings
cold_start_recommendations = cold_start_recommender.recommend_for_new_user(
    user_demographics=new_user_demographics,
    user_ratings=None,  # No ratings yet
    num_recommendations=10
)

print(f"\nTop 10 Cold Start Recommendations:")
for i, (movie_id, title, score) in enumerate(cold_start_recommendations, 1):
    print(f"{i:2d}. {title:<50} (Score: {score:.3f})")

# Example 2: Get onboarding movies for initial rating collection
print(f"\n\n2. ONBOARDING MOVIES FOR RATING COLLECTION")
print("-" * 50)

onboarding_movies = cold_start_recommender.get_onboarding_movies(num_movies=8)

print("Movies to show new user for initial ratings (diverse genres):")
for i, (movie_id, title, genres) in enumerate(onboarding_movies, 1):
    print(f"{i}. {title:<40} | Genres: {genres}")

# Example 3: Warm Cold Start - User has provided some initial ratings
print(f"\n\n3. WARM COLD START SCENARIO")
print("-" * 50)

# Simulate user rating some of the onboarding movies
initial_ratings = [
    (onboarding_movies[0][0], 5),  # Loved the first movie
    (onboarding_movies[1][0], 4),  # Liked the second movie
    (onboarding_movies[2][0], 2),  # Didn't like the third movie
    (onboarding_movies[3][0], 4),  # Liked the fourth movie
]

print("User's initial ratings:")
for movie_id, rating in initial_ratings:
    movie_title = movies[movies['movie_id'] == movie_id]['title'].iloc[0]
    print(f"  {movie_title:<50} - Rating: {rating}/5")

# Get improved recommendations based on initial ratings
warm_recommendations = cold_start_recommender.recommend_for_new_user(
    user_demographics=new_user_demographics,
    user_ratings=initial_ratings,
    num_recommendations=10
)

print(f"\nTop 10 Recommendations after initial ratings:")
for i, (movie_id, title, score) in enumerate(warm_recommendations, 1):
    print(f"{i:2d}. {title:<50} (Score: {score:.3f})")

print(f"\n\n4. COMPARISON: Different User Demographics")
print("-" * 50)

# Example with different demographics
female_user_demographics = {
    'gender': 'F',     # Female
    'age': 45,         # 45 years old
    'occupation': 0    # Other/not specified
}

print(f"Different User Demographics: {female_user_demographics}")

female_recommendations = cold_start_recommender.recommend_for_new_user(
    user_demographics=female_user_demographics,
    user_ratings=None,
    num_recommendations=5
)

print(f"\nTop 5 Recommendations for different demographic:")
for i, (movie_id, title, score) in enumerate(female_recommendations, 1):
    print(f"{i}. {title:<50} (Score: {score:.3f})")

print(f"\n{'='*80}")
print("Cold Start Recommendation Demonstration Complete!")
print(f"{'='*80}")


COLD START RECOMMENDATION SYSTEM DEMONSTRATION

1. PURE COLD START SCENARIO
--------------------------------------------------
New User Demographics: {'gender': 'M', 'age': 25, 'occupation': 4}

Top 10 Cold Start Recommendations:
 1. Life Is Beautiful (La Vita � bella) (1997)         (Score: 0.997)
 2. Sling Blade (1996)                                 (Score: 0.992)
 3. Maltese Falcon, The (1941)                         (Score: 0.989)
 4. Platoon (1986)                                     (Score: 0.988)
 5. Doctor Zhivago (1965)                              (Score: 0.988)
 6. Rebel Without a Cause (1955)                       (Score: 0.985)
 7. Vertigo (1958)                                     (Score: 0.983)
 8. Akira (1988)                                       (Score: 0.974)
 9. Fisher King, The (1991)                            (Score: 0.970)
10. Third Man, The (1949)                              (Score: 0.968)


2. ONBOARDING MOVIES FOR RATING COLLECTION
-------------------------